In [ ]:
# 기사 임베딩 적용해보기

In [2]:
## 라이브러리 로드
from IPython.display import display,HTML
display(HTML("<style>.container {width:95% !important ; }</style>"))
from pathlib import Path
import pandas as pd
import numpy as np
import pickle
import re
import json
from collections import Counter
import itertools
from datetime import datetime, timedelta
import time
import calendar
from matplotlib import font_manager as fm
import matplotlib.pyplot as plt
from matplotlib import rc

### 기사 클러스터링 및 감성분석에 사용할 후보 임베딩 모델(PyTorch 호환가능)
#### KoBERT
    - 특히 감성 분석에 강한 모델로, 한국어의 뉘앙스를 잘 파악하고 다양한 감정 상태를 분류하는 데 유용
    - 클러스터링 작업에도 임베딩의 품질이 좋기 때문에 적합

#### KR-BERT
    - 다양한 한국어 텍스트를 높은 정확도로 처리할 수 있는 모델
    - 특히 세밀한 언어적 특성을 이해할 필요가 있는 감성 분석에 적합하며 클러스터링에서도 효과적인 임베딩을 제공

#### KoELECTRA
    - 이 모델은 특히 감성 분석에서 고성능을 보여주며, 미묘한 문맥상의 차이를 포착
    - 클러스터링을 위한 특징 추출에서도 우수한 성능을 발휘할 수 있음.

In [3]:
# 작업경로
path = "C:/Users/master/Documents/kjh/issue_trends/"
ym = '202403'


In [4]:
df_raw_data = pd.read_csv(path+"/input_data/news_{}.csv".format(ym), encoding = "utf-8-sig", 
                          usecols = ['date', 'journal', 'sid2_name', 'title', 'content', 'href'])

,journal,sid2_name,date,title,content,href
0,중앙일보,경제 일반,20240331,차종 다변화 전략 통했다…1~2월 국산차 수출 100억 달러 돌파,경기도 평택시 포승읍 평택항의 현대차 수출 부두. 전민규 기자 수출 차종을 다변화하...,https://n.news.naver.com/mnews/article/025/000...
1,한국일보,산업/재계,20240331,아마존·소니·구글 문 연달아 두드린 택진이형...글로벌 행보 본격화,"김택진 엔씨소프트 대표, 빅테크 회사들과 '맞손' 김택진·박병무 공동대표 체제 28...",https://n.news.naver.com/mnews/article/469/000...
2,한국경제,산업/재계,20240331,"SKT, 창사 40주년 맞이 서울숲 환경 정화 봉사활동",SK텔레콤은 지난 29일 창사 40주년을 맞아 유영상 사장을 비롯한 전사 임직원 약...,https://n.news.naver.com/mnews/article/015/000...
3,MBC,경제 일반,20240331,"금감원, '자유적금계좌 악용 중고거래 사기'에 소비자경보",사진제공:연합뉴스 금융감독원은 자유적금계좌를 악용한 중고거래 사기가 성행하고 있어 ...,https://n.news.naver.com/mnews/article/214/000...
4,MBC,경제 일반,20240331,7월부터 군장병 복무기간 실손보험료 납입 중지 가능,오는 7월부터 군장병은 복무기간에 실손보험료 납입을 중지했다가 복무가 끝나면 재개할...,https://n.news.naver.com/mnews/article/214/000...
...,...,...,...,...,...,...
19077,서울신문,경제 일반,20240301,"“우주 비행사 체험하세요”…LGU+ 키즈토피아, 우주 체험 메타버스 공간 만든다",애니메이션 ‘아스트로 스테이션’ 메타버스 콘텐츠 애니메이션 배경과 같은 가상 공간 ...,https://n.news.naver.com/mnews/article/081/000...
19078,세계일보,경제 일반,20240301,"삼프로tv, 코스닥 상장 불발",증권전문 유투브 채널 삼프로TV의 코스닥 상장이 불발됐다. 1일 한국거래소 등...,https://n.news.naver.com/mnews/article/022/000...
19079,한겨레,금융,20240301,미국 1월 물가 예상에 부합…연준 인사 “여름에 금리 인하 적절”,미국 연방준비제도 건물 전경. 로이터 연합뉴스 올해 1월 미국 소비자물가가 1년 전...,https://n.news.naver.com/mnews/article/028/000...
19080,KBS,경제 일반,20240301,[간추린 뉴스] 대중 무역수지 17개월 만에 흑자 외,간추린 뉴스입니다. 대중 무역수지 17개월 만에 흑자 지난달 대중국 무역수지가 2억...,https://n.news.naver.com/mnews/article/056/001...


In [6]:
from transformers import AutoModel, AutoTokenizer

# 모델과 토크나이저 불러오기
model = AutoModel.from_pretrained("monologg/kobert")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

In [10]:
data = data.fillna("N/A")  # None을 'N/A'로 변환
data = data.apply(lambda x: str(x) if not isinstance(x, str) else x) 

# 텍스트를 토큰화하고 텐서로 변환
inputs = tokenizer(list(df_raw_data['content']), return_tensors="pt", padding=True, truncation=True)


# 모델을 통해 임베딩 추출
outputs = model(**inputs)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [8]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3605,  0.0266,  0.0375,  ...,  0.2321, -0.2777, -0.3355],
         [ 0.1337,  0.0116, -0.0511,  ...,  0.1821, -0.3296,  0.0527],
         [ 0.4276, -0.0075,  0.1248,  ..., -0.1380, -0.4215,  0.0422],
         [ 0.3691,  0.1383, -0.0291,  ..., -0.0616, -0.3605, -0.0590],
         [ 0.3262,  0.1804,  0.3501,  ...,  0.0910, -0.3550, -0.0826],
         [ 0.2153, -0.0169,  0.0059,  ..., -0.1945, -0.6010,  0.1805]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 4.2551e-02, -2.0242e-03, -1.2560e-01, -7.9353e-03,  6.1880e-01,
          6.7225e-01, -9.0249e-01,  1.3304e-01,  6.4012e-02, -4.8637e-02,
          4.4609e-01,  3.1665e-02, -8.5247e-02,  1.9384e-02,  4.6222e-02,
         -3.4532e-01, -4.9552e-01,  7.3623e-02,  1.4105e-02, -1.3562e-02,
          6.6050e-02, -1.2922e-02, -2.2028e-02, -2.4932e-01,  6.3370e-02,
         -3.1419e-01,  2.7788e-01, -3.2650e-02,  9.4582e-01, -1.1881e-01,
       